In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats,
                                    leaguedashplayerptshot)
import time

In [2]:
current_year="2023-24"

In [3]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/"+current_year+"_pbp.csv")

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [4]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season=current_year,season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season=current_year,
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [5]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [6]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
264,Jaylin Williams,0.857143
50,Brandin Podziemski,0.714607
411,Moritz Wagner,0.643575
178,Garrison Mathews,0.532394
203,Isaiah Joe,0.508696
486,Sam Merrill,0.505618
329,Kenrich Williams,0.500000
230,Jalen Brunson,0.430769
354,Kyle Lowry,0.418310
522,Tim Hardaway Jr.,0.392727


# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
303,Josh Okogie,1.746746
531,Trendon Watford,1.650000
374,Malachi Flynn,1.586441
459,Paul Reed,1.566839
506,T.J. McConnell,1.500000
413,Moses Moody,1.486047
225,Jae'Sean Tate,1.319255
480,Russell Westbrook,1.307623
136,Dennis Smith Jr.,1.295238
13,Alex Caruso,1.251064


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
394,Matisse Thybulle,5.313537
13,Alex Caruso,4.723404
163,Dyson Daniels,4.562212
136,Dennis Smith Jr.,4.495238
349,Kris Dunn,4.155319
506,T.J. McConnell,3.780000
152,Donte DiVincenzo,3.728571
496,Shai Gilgeous-Alexander,3.725581
325,Kelly Olynyk,3.681818
459,Paul Reed,3.655959


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [9]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
339,Kevon Looney,14.923636
149,Domantas Sabonis,14.460335
249,James Wiseman,14.221118
161,Dwight Powell,14.100000
426,Nick Richards,13.425564
528,Trayce Jackson-Davis,13.253503
317,Jusuf Nurkic,13.018450
232,Jalen Duren,12.945455
110,Daniel Gafford,12.636437
123,Day'Ron Sharpe,12.373510


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [10]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [11]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [12]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
376,Malcolm Brogdon,0.198830,0.067251,0.295322,0.172515,0.131579,0.163743,1.029240
402,Michael Porter Jr.,0.198830,0.421053,0.125731,0.081871,0.190058,0.023392,1.040936
375,Malaki Branham,0.198830,0.149123,0.157895,0.032164,0.064327,0.473684,1.076023
345,Klay Thompson,0.198830,0.342105,0.210526,0.125731,0.207602,0.102339,1.187135
514,Terance Mann,0.438596,0.292398,0.040936,0.143275,0.318713,0.146199,1.380117
279,Joe Ingles,0.198830,0.005848,0.038012,0.520468,0.350877,0.277778,1.391813
26,Anfernee Simons,0.461988,0.137427,0.181287,0.040936,0.011696,0.573099,1.406433
383,Marcus Morris Sr.,0.198830,0.222222,0.058480,0.111111,0.728070,0.099415,1.418129
502,Spencer Dinwiddie,0.198830,0.429825,0.365497,0.166667,0.210526,0.049708,1.421053
166,Eric Gordon,0.198830,0.043860,0.017544,0.441520,0.160819,0.593567,1.456140


In [13]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
455,Patrick Beverley,0.894737,0.152047,0.444444,0.681287,0.017544,0.804094,2.994152
480,Russell Westbrook,0.839181,0.154971,0.064327,0.842105,0.043860,0.923977,2.868421


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [14]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
264,Jaylin Williams,1.000000,0.903509,0.959064,0.581871,0.950292,0.862573,5.257310
225,Jae'Sean Tate,0.961988,0.646199,0.985380,0.801170,0.771930,0.961988,5.128655
90,Cody Zeller,0.991228,0.891813,0.485380,0.611111,0.997076,0.903509,4.880117
196,Herbert Jones,0.918129,0.564327,0.988304,0.871345,0.692982,0.786550,4.821637
161,Dwight Powell,0.760234,0.976608,0.836257,0.663743,0.932749,0.646199,4.815789
21,Andre Drummond,0.697368,0.798246,0.938596,0.929825,0.769006,0.619883,4.752924
155,Draymond Green,0.988304,0.827485,0.991228,0.786550,0.912281,0.239766,4.745614
411,Moritz Wagner,0.994152,0.789474,0.564327,0.684211,0.973684,0.614035,4.619883
459,Paul Reed,0.198830,0.941520,0.792398,0.953216,0.868421,0.865497,4.619883
487,Sandro Mamukelashvili,0.198830,0.807018,0.982456,0.692982,0.959064,0.953216,4.593567


# The Trickshot Grenadier Award (presented by Dude Perfect)

Highest sum of percentile ranks in FGA, FGA frequency & eFG% on shots with 4 seconds or less on the shotclock (minimum 50th percentile in FGA) (credit to BehavioralSink & Bylanta for the idea)

In [15]:
late_shotclock_shots=leaguedashplayerptshot.LeagueDashPlayerPtShot(shot_clock_range_nullable="4-0 Very Late").league_dash_ptshots.get_data_frame().loc[:, 'PLAYER_ID':'EFG_PCT']
late_shotclock_shots["FGA_percentile"]=late_shotclock_shots.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile=late_shotclock_shots.query("FGA_percentile>=0.5").copy()
late_shotclock_shots_top_50_percentile["FGA_percentile_among_top_50"]=late_shotclock_shots_top_50_percentile.FGA.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["FGA_FREQ_percentile"]=late_shotclock_shots_top_50_percentile.FGA_FREQUENCY.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["EFG_percentile"]=late_shotclock_shots_top_50_percentile.EFG_PCT.rank(method='min',pct=True)
late_shotclock_shots_top_50_percentile["avg_percentile"]=late_shotclock_shots_top_50_percentile[['FGA_FREQ_percentile','FGA_percentile','EFG_percentile']].sum(axis=1)
late_shotclock_shots_top_50_percentile.nlargest(n=10,columns="avg_percentile")

,PLAYER_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,AGE,GP,G,FGA_FREQUENCY,FGM,FGA,FG_PCT,EFG_PCT,FGA_percentile,FGA_percentile_among_top_50,FGA_FREQ_percentile,EFG_percentile,avg_percentile
28,1629636,Darius Garland,1610612739,CLE,24.0,51,43,0.133,42,99,0.424,0.520,0.945525,0.889764,0.877953,0.830709,2.654187
0,203999,Nikola Jokic,1610612743,DEN,29.0,70,61,0.133,74,165,0.448,0.488,1.000000,1.000000,0.877953,0.748031,2.625984
2,202695,Kawhi Leonard,1610612746,LAC,32.0,67,58,0.116,59,132,0.447,0.519,0.992218,0.984252,0.799213,0.826772,2.618202
29,202699,Tobias Harris,1610612755,PHI,31.0,66,44,0.109,46,98,0.469,0.536,0.937743,0.874016,0.724409,0.874016,2.536168
110,1627936,Alex Caruso,1610612741,CHI,30.0,65,35,0.112,28,55,0.509,0.691,0.778210,0.551181,0.755906,0.992126,2.526242
69,1629130,Duncan Robinson,1610612748,MIA,29.0,63,46,0.108,29,70,0.414,0.564,0.863813,0.724409,0.716535,0.925197,2.505546
239,1631367,Jacob Gilyard,1610612751,BKN,25.0,39,20,0.185,12,27,0.444,0.630,0.527237,0.043307,0.992126,0.980315,2.499678
76,200768,Kyle Lowry,1610612755,PHI,38.0,54,31,0.188,24,66,0.364,0.462,0.848249,0.692913,0.996063,0.653543,2.497855
197,1626204,Larry Nance Jr.,1610612740,NOP,31.0,55,28,0.158,17,33,0.515,0.561,0.607004,0.204724,0.952756,0.909449,2.469209
12,1630532,Franz Wagner,1610612753,ORL,22.0,66,53,0.113,48,114,0.421,0.482,0.976654,0.952756,0.763780,0.724409,2.464843


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked where the blocker is at least 5 inches shorter than the blockee

In [16]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
80,Fred VanVleet,40
67,Derrick White,25
154,Luguentz Dort,19
30,CJ McCollum,15
50,D'Angelo Russell,14
85,Grayson Allen,13
148,Kyrie Irving,13
74,Donovan Mitchell,12
110,James Harden,12
107,Jalen Williams,11


# The "Pick On Someone Your Own Size" Award

most shots blocked where the blocker is at least 5 inches taller than the blockee

In [17]:
blocking_player_heights.query('height_difference <= -5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
232,Victor Wembanyama,193
25,Brook Lopez,116
32,Chet Holmgren,112
205,Rudy Gobert,88
234,Walker Kessler,87
146,Kristaps Porzingis,74
176,Nic Claxton,69
11,Anthony Davis,61
58,Dereck Lively II,61
96,Jakob Poeltl,56


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [18]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
100,Tyus Jones,0.061
465,Marcus Sasser,0.066
219,Trae Young,0.081
156,Luke Kennard,0.085
508,Keyonte George,0.091
271,Jordan Poole,0.093
312,Immanuel Quickley,0.096
520,Nick Smith Jr.,0.102
195,Aaron Holiday,0.103
73,Seth Curry,0.106


alternatively: restricting to players > 6 foot 6 inches in height

In [19]:
above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
79,Jayson Tatum,0.183
33,Dalen Terry,0.184
72,Jalen McDaniels,0.185
154,Sam Hauser,0.185
86,Joe Ingles,0.187
47,Duncan Robinson,0.191
23,Cameron Johnson,0.197
105,Kevin Huerter,0.199
5,Amir Coffey,0.204
165,Trey Murphy III,0.206


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [20]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
311,Zeke Nnaji,0.581
173,Luke Kornet,0.564
368,Day'Ron Sharpe,0.533
313,Paul Reed,0.532
16,JaVale McGee,0.514
169,Thomas Bryant,0.508
364,Isaiah Jackson,0.507
446,Walker Kessler,0.501
164,Isaiah Hartenstein,0.498
473,Trayce Jackson-Davis,0.496


alternatively: restricting to players < 6 foot 7 inches in height

In [21]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
98,Josh Okogie,0.453
105,Kenrich Williams,0.415
76,Jaden Springer,0.406
136,Ochai Agbaji,0.378
166,Zion Williamson,0.374
78,Jae'Sean Tate,0.371
3,Aaron Wiggins,0.370
66,Grant Williams,0.354
58,Draymond Green,0.350
63,Gary Payton II,0.350
